For this chapter we'll be participating in a real kaggle competition and evaluate each of the regularization techniques of the chapter 7                                                                                                               
https://www.kaggle.com/c/rsna-str-pulmonary-embolism-detection/overview                                                       
dataset from                                                                                                                  
https://www.kaggle.com/c/rsna-str-pulmonary-embolism-detection/discussion/182930

In [1]:
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pylab as plt
import os
from PIL import Image

import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
import torchvision.models as models
import torch.nn as nn
import torch.nn.functional as F

import torch.optim as optim

# import cv2
import glob

In [2]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
device
    
device

device(type='cuda')

In [3]:
train_path = 'D:\\lungs\\train-jpegs'
csv_path = 'E:\\Download\\new_downloads\\train.csv'
# sample_subm_csv_path = 'E:Download\\new_downloads\\sample_submission.csv'

In [4]:
df_train = pd.read_csv(csv_path)

In [5]:
df_train.shape

(1790594, 17)

In [6]:
df_train.head()

,StudyInstanceUID,SeriesInstanceUID,SOPInstanceUID,pe_present_on_image,negative_exam_for_pe,qa_motion,qa_contrast,flow_artifact,rv_lv_ratio_gte_1,rv_lv_ratio_lt_1,leftsided_pe,chronic_pe,true_filling_defect_not_pe,rightsided_pe,acute_and_chronic_pe,central_pe,indeterminate
0,6897fa9de148,2bfbb7fd2e8b,c0f3cb036d06,0,0,0,0,0,0,1,1,0,0,1,0,0,0
1,6897fa9de148,2bfbb7fd2e8b,f57ffd3883b6,0,0,0,0,0,0,1,1,0,0,1,0,0,0
2,6897fa9de148,2bfbb7fd2e8b,41220fda34a3,0,0,0,0,0,0,1,1,0,0,1,0,0,0
3,6897fa9de148,2bfbb7fd2e8b,13b685b4b14f,0,0,0,0,0,0,1,1,0,0,1,0,0,0
4,6897fa9de148,2bfbb7fd2e8b,be0b7524ffb4,0,0,0,0,0,0,1,1,0,0,1,0,0,0


In [7]:
df_train.tail()

,StudyInstanceUID,SeriesInstanceUID,SOPInstanceUID,pe_present_on_image,negative_exam_for_pe,qa_motion,qa_contrast,flow_artifact,rv_lv_ratio_gte_1,rv_lv_ratio_lt_1,leftsided_pe,chronic_pe,true_filling_defect_not_pe,rightsided_pe,acute_and_chronic_pe,central_pe,indeterminate
1790589,4833c9b6a5d0,57e3e3c5f910,da0ecef50cf5,0,0,0,0,0,0,1,1,1,0,0,0,0,0
1790590,4833c9b6a5d0,57e3e3c5f910,d74b46c2f2c4,0,0,0,0,0,0,1,1,1,0,0,0,0,0
1790591,4833c9b6a5d0,57e3e3c5f910,ba71189191ad,0,0,0,0,0,0,1,1,1,0,0,0,0,0
1790592,4833c9b6a5d0,57e3e3c5f910,f4fdc88f2ace,0,0,0,0,0,0,1,1,1,0,0,0,0,0
1790593,4833c9b6a5d0,57e3e3c5f910,f890efd48940,1,0,0,0,0,0,1,1,1,0,0,0,0,0


In [8]:
image_level_features = ['pe_present_on_image']
# exam_level_features = ['negative_exam_for_pe', 
exam_level_features = ['indeterminate', 'chronic_pe', 'acute_and_chronic_pe', 'central_pe', 'leftsided_pe', 'rightsided_pe', 'rv_lv_ratio_gte_1', 'rv_lv_ratio_lt_1',]
informational_features = ['qa_motion', 'qa_contrast', 'true_filling_defect_not_pe', 'flow_artifact']

In [9]:
df_train.loc[df_train['pe_present_on_image'] == 0, exam_level_features] = 0

In [10]:
df_train = df_train[df_train.pe_present_on_image == 1].append(df_train[df_train.pe_present_on_image == 0].sample(df_train[df_train.pe_present_on_image == 1].shape[0]), ignore_index=True).sample(frac=1)
df_train = df_train[df_train.pe_present_on_image == 1].append(df_train[df_train.pe_present_on_image == 0].sample(df_train[df_train.pe_present_on_image == 1].shape[0]), ignore_index=True)

df_train = df_train.sample(frac=1).reset_index(drop=True)

In [11]:
df_train.head()

,StudyInstanceUID,SeriesInstanceUID,SOPInstanceUID,pe_present_on_image,negative_exam_for_pe,qa_motion,qa_contrast,flow_artifact,rv_lv_ratio_gte_1,rv_lv_ratio_lt_1,leftsided_pe,chronic_pe,true_filling_defect_not_pe,rightsided_pe,acute_and_chronic_pe,central_pe,indeterminate
0,ddcd4deceb98,62929b5bf178,82ddcf09683f,0,1,0,0,0,0,0,0,0,0,0,0,0,0
1,84e494d67539,5f10c2bb4bf2,de2f7f8f4c38,1,0,0,0,0,0,1,1,1,0,0,0,0,0
2,b414e0ac7cec,d9b33900f2b4,1d959d9dd76f,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,9f03e1e55791,76ed03625431,b854d125dd76,1,0,0,0,0,1,0,1,0,0,1,0,1,0
4,4b897f593782,d4dbb09b2897,d84d63a50be0,1,0,0,0,0,1,0,1,0,0,1,0,0,0


In [ ]:
df_train.tail()

In [ ]:
# ims__ = []
# for i, c in enumerate(df_train.StudyInstanceUID.unique()):
#     print(i)
#     ims__.append(len(df_train[df_train.StudyInstanceUID == c]))

In [ ]:
min(ims__)

In [ ]:
# sample_csv_df = pd.read_csv(sample_subm_csv_path)

In [ ]:
def get_an_image(df, idx=0):
    row = df.iloc[idx]
#     print('Reading image ', glob.glob(f"{train_path}/{row[0]}/{row[1]}/*{row[2]}.jpg"))
    print('Reading image ', glob.glob(f"{train_path}/{row[0]}/{row[1]}/*{row[2]}.jpg")[0])
    img = Image.open(glob.glob(f"{train_path}/{row[0]}/{row[1]}/*{row[2]}.jpg")[0])
    return img

In [ ]:
# def get_an_image(df, idx=0):
#     row = df.iloc[idx]
# #     print('Reading image ', glob.glob(f"{train_path}/{row[0]}/{row[1]}/*{row[2]}.jpg"))
#     print('Reading image ', glob.glob(f"{train_path}/{row[0]}/{row[1]}/*{row[2]}.jpg")[0])
#     img = Image.open(glob.glob(f"{train_path}/{row[0]}/{row[1]}/*{row[2]}.jpg")[0])
#     return img

In [ ]:
img = get_an_image(df_train, 0)

In [ ]:
np.array(img).shape

In [ ]:
plt.imshow(img)

In [ ]:
def plot_image(img):
    plt.figure(figsize=[12,6])
    plt.subplot(131)
    plt.imshow(img[:,:,0],cmap='gray')
    plt.subplot(132)
    plt.imshow(img[:,:,1],cmap='gray')
    plt.subplot(133)
    plt.imshow(img[:,:,2],cmap='gray')

In [ ]:
plot_image(np.array(img))

In [ ]:
# study_instances =  pd.read_csv(csv_path).StudyInstanceUID.unique()
# len(study_instances)

# len(pd.read_csv(csv_path).StudyInstanceUID.unique())

In [ ]:
len(df_train.loc[df_train.StudyInstanceUID == study_instances[0]])

In [ ]:
df_train.loc[df_train.StudyInstanceUID == study_instances[0]].sample(100)

In [ ]:
image_level_features

In [ ]:
class LungsDataset(Dataset):

    def __init__(self, csv_file, transform=None):
#         self.sample_size = 32 # min unique
#         self.sample_size = 1
        self.csv_file = csv_file
#         self.study_instances = self.csv_file.StudyInstanceUID.unique()
        self.transform = transform

    def __len__(self):
        return len(self.csv_file)

    def __getitem__(self, idx):
#         sub_df = self.csv_file.loc[self.csv_file.StudyInstanceUID == self.study_instances[idx]].sample(self.sample_size)
#         imgs = []
# #         print(sub_df)
#         for i in range(len(sub_df)):
#             print(i)
# #             print(sub_df)
#             img_ = get_an_image(sub_df, i)
#             if self.transform:
#                 img_ = self.transform(img_).to(device)
#                 imgs.append(img_)
            
#         targets = torch.FloatTensor(sub_df.loc[sub_df.StudyInstanceUID == self.study_instances[idx]][image_level_features+exam_level_features+informational_features].values).to(device)
#         targets = torch.autograd.Variable((sub_df[image_level_features+exam_level_features + informational_features].values)).to(device)
#         print(targets.shape, imgs[0].shape, len(imgs))
        row = self.csv_file.iloc[idx]
        img = Image.open(glob.glob(f"{train_path}/{row[0]}/{row[1]}/*{row[2]}.jpg")[0])
        img = self.transform(img).to(device)
        targets = torch.tensor(row[image_level_features+exam_level_features + informational_features].values.astype(np.float16)).float().to(device)
#         print(targets.shape)
        return torch.autograd.Variable(img), targets
#         return torch.autograd.Variable(torch.stack(imgs)), targets

In [ ]:
normalize = transforms.Normalize(
    mean=[0.485, 0.456, 0.406],
    std=[0.229, 0.224, 0.225]
)
# normalize = transforms.Normalize(
#     mean=[0.5, 0.5, 0.5],
#     std=[0.5, 0.5, 0.5]
# )

ds_trans = transforms.Compose([
                                transforms.Grayscale(num_output_channels=3),
                                transforms.Resize((224, 224)),
                                transforms.ToTensor(),
                                
#                                 transforms.CenterCrop(224),
                                normalize
])

invTrans = transforms.Compose([ transforms.Normalize(mean = [ 0., 0., 0. ],
                                                     std = [ 1/0.229, 1/0.224, 1/0.225 ]),
                                transforms.Normalize(mean = [ -0.485, -0.456, -0.406 ],
                                                     std = [ 1., 1., 1. ]),
                               ])

In [ ]:
train_dataset = LungsDataset(df_train, transform=ds_trans)

len(train_dataset)

In [ ]:
train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True)

In [ ]:
# xx = 0
# yy = 0
# for i, x in enumerate(train_dataloader):
# #     print('AAAAAAAAA', x[0].shape, x[1].shape)
# #     print(x[0])
    
#     print('---'*40)
#     if i == 0:
#         xx = x[0]
#     if i == 1:
#         yy = x[1]
#         break
# #     print(x[1])

In [ ]:
# len(x)

In [ ]:
# xx.shape, yy.shape

In [ ]:
# plot_image(xx[0].cpu().permute(1,2,0))

In [ ]:
# yy[3].shape

In [ ]:
# plot_image(xx[3].cpu().permute(1,2,0))

In [ ]:
# df_train.columns

In [ ]:
resnet101 = models.resnet50()
fc_feat_size = resnet101.fc.in_features
resnet101.fc.in_features, resnet101.fc.out_features
# fc_feat_size = 2048
# resnet50.fc = nn.Linear(resnet50.fc.in_features, fc_feat_size)

In [ ]:
class Flatten(nn.Module):
    def __init__(self):
        super(Flatten, self).__init__()
    def __forward__(self, x):
        x = x.view(x.size(0), -1)
        return x
    
class Identity(nn.Module):
    def __init__(self):
        super(Identity, self).__init__()
    def forward(self, x):
        return x

In [ ]:
resnet101.avgpool = Identity()
resnet101.fc = Identity()

In [ ]:
resnet101

In [ ]:
image_level_features+exam_level_features+informational_features

In [ ]:
fc_feat_size

In [ ]:
# fc_feat_size = 1024
def weights_init(m):
    classname = m.__class__.__name__
    for l in m.modules():
        if isinstance(l, nn.Conv2d):
            torch.nn.init.uniform_(l.weight)
        elif isinstance(l, nn.BatchNorm2d):
            torch.nn.init.uniform_(l.weight)
        elif isinstance(l, nn.Linear):
            torch.nn.init.xavier_normal_(l.weight)
        
class MultiTaskHead(nn.Module):
    def __init__(self):
        super(MultiTaskHead, self).__init__()
        self.m = nn.Sequential(
                nn.Conv2d(2048, 512, kernel_size=(1,1), stride=(1,1), bias=False), 
                nn.BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
                nn.Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False),
                nn.BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
                nn.Conv2d(512, 2048, kernel_size=(1, 1), stride=(1, 1), bias=False),
                nn.BatchNorm2d(2048, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
                nn.AdaptiveAvgPool2d(output_size=(1,1)),
        )
        self.l = nn.Sequential(
                nn.Linear(fc_feat_size, 1, bias=True), 
#                 nn.ReLU(),
#                 nn.Linear(512, 1), 
                nn.Sigmoid()
        )
        
    def forward(self, x):
        x = x.view(x.shape[0], 2048, int(np.sqrt(x.shape[1]/2048)), int(np.sqrt(x.shape[1]/2048)))
        x = self.m(x)
        x = self.l(x.squeeze())
        return x

class WrapperModel(nn.Module):
    def __init__(self, pretrained_model):
        super(WrapperModel, self).__init__()
        self.stage = 'train'
        
        self.backbone = pretrained_model
        '''
        image-level feature
        '''
        self.linear_pe_present_on_image = MultiTaskHead()
        weights_init(self.linear_pe_present_on_image)
        '''
        exam-level features
        '''
        self.linear_negative_exam_for_pe = MultiTaskHead()
        weights_init(self.linear_negative_exam_for_pe)
        
        self.linear_indeterminate = MultiTaskHead()
        weights_init(self.linear_indeterminate)
        
        self.linear_chronic_pe = MultiTaskHead()
        weights_init(self.linear_chronic_pe)
        
        self.linear_acute_and_chronic_pe = MultiTaskHead()
        weights_init(self.linear_acute_and_chronic_pe)
        
        self.linear_central_pe = MultiTaskHead()
        weights_init(self.linear_central_pe)
        
        self.linear_leftsided_pe = MultiTaskHead()
        weights_init(self.linear_leftsided_pe)
        
        self.linear_rightsided_pe = MultiTaskHead()
        weights_init(self.linear_rightsided_pe)
        
        self.linear_rv_lv_ratio_gte_1 = MultiTaskHead()
        weights_init(self.linear_rv_lv_ratio_gte_1)
        
        self.linear_rv_lv_ratio_lt_1 = MultiTaskHead()
        weights_init(self.linear_rv_lv_ratio_lt_1)
        
        '''
        informational features
        '''
        self.linear_qa_motion = MultiTaskHead()
        weights_init(self.linear_qa_motion)
        
        self.linear_qa_contrast = MultiTaskHead()
        weights_init(self.linear_qa_contrast)
                
        self.linear_true_filling_defect_not_pe = MultiTaskHead()
        weights_init(self.linear_true_filling_defect_not_pe)
        
        self.linear_flow_artifact = MultiTaskHead()
        weights_init(self.linear_flow_artifact)

    def forward(self, x):
#         if self.stage == 'train':
#             x, _ = self.backbone(x)
#         else: 
        x = self.backbone(x)
        
#         print('x1 .shape', x1.shape)
        x_pe_present_on_image = self.linear_pe_present_on_image(x)
        
        x_negative_exam_for_pe = self.linear_negative_exam_for_pe(x)
        x_indeterminate = self.linear_indeterminate(x)
        x_chronic_pe = self.linear_chronic_pe(x)
        x_acute_and_chronic_pe = self.linear_acute_and_chronic_pe(x)
        x_central_pe = self.linear_central_pe(x)
        x_leftsided_pe = self.linear_leftsided_pe(x)
        x_rightsided_pe = self.linear_rightsided_pe(x)
        x_rv_lv_ratio_gte_1 = self.linear_rv_lv_ratio_gte_1(x)
        x_rv_lv_ratio_lt_1 = self.linear_rv_lv_ratio_lt_1(x)
        
        x_qa_motion = self.linear_qa_motion(x)
        x_qa_contrast = self.linear_qa_contrast(x)
        x_true_filling_defect_not_pe = self.linear_true_filling_defect_not_pe(x)
        x_flow_artifact = self.linear_flow_artifact(x)
        
#         ys = []
#         for y_pred in [ x_pe_present_on_image, x_negative_exam_for_pe, x_indeterminate, x_chronic_pe, x_acute_and_chronic_pe, x_central_pe, x_leftsided_pe, x_rightsided_pe, x_rv_lv_ratio_gte_1, x_rv_lv_ratio_lt_1, x_qa_motion, x_qa_contrast, x_true_filling_defect_not_pe, x_flow_artifact]:
#             ys.append(y_pred)
        ys = []
        for y_pred in [ x_pe_present_on_image, x_indeterminate, x_chronic_pe, x_acute_and_chronic_pe, x_central_pe, x_leftsided_pe, x_rightsided_pe, x_rv_lv_ratio_gte_1, x_rv_lv_ratio_lt_1, x_qa_motion, x_qa_contrast, x_true_filling_defect_not_pe, x_flow_artifact]:
            ys.append(y_pred)
        return torch.cat(ys, 1)
                      

model = WrapperModel(resnet101).to(device)

In [ ]:
# # fc_feat_size = 1024
# def weights_init(m):
#     classname = m.__class__.__name__
#     for l in m.modules():
#         if isinstance(l, nn.Conv2d):
#             torch.nn.init.uniform_(l.weight)
#         elif isinstance(l, nn.BatchNorm2d):
#             torch.nn.init.uniform_(l.weight)
#         elif isinstance(l, nn.Linear):
#             torch.nn.init.xavier_normal_(l.weight)
        
# class MultiTaskHead(nn.Module):
#     def __init__(self):
#         super(MultiTaskHead, self).__init__()
#         self.m = nn.Sequential(
#                 nn.Conv2d(2048, 512, kernel_size=(1,1), stride=(1,1), bias=False), 
#                 nn.BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
#                 nn.Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False),
#                 nn.BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
#                 nn.Conv2d(512, 2048, kernel_size=(1, 1), stride=(1, 1), bias=False),
#                 nn.BatchNorm2d(2048, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
#                 nn.AdaptiveMaxPool2d(output_size=(1,1)),
#         )
#         self.l = nn.Sequential(
#                 nn.Linear(fc_feat_size, 1), 
# #                 nn.ReLU(),
# #                 nn.Linear(512, 1), 
#                 nn.Sigmoid()
#         )
        
#     def forward(self, x):
#         x = x.view(x.shape[0], 2048, int(np.sqrt(x.shape[1]/2048)), int(np.sqrt(x.shape[1]/2048)))
#         x = self.m(x)
#         x = self.l(x.squeeze())
#         return x

# class WrapperModel(nn.Module):
#     def __init__(self, pretrained_model):
#         super(WrapperModel, self).__init__()
        
#         self.backbone = pretrained_model
#         '''
#         image-level feature
#         '''
#         self.linear_pe_present_on_image = nn.Sequential(nn.Linear(2048,1), nn.Sigmoid())
#         weights_init(self.linear_pe_present_on_image)
#         '''
#         exam-level features
#         '''
#         self.linear_negative_exam_for_pe = nn.Sequential(nn.Linear(2048,1), nn.Sigmoid())
#         weights_init(self.linear_negative_exam_for_pe)
        
#         self.linear_indeterminate = nn.Sequential(nn.Linear(2048,1), nn.Sigmoid())
#         weights_init(self.linear_indeterminate)
        
#         self.linear_chronic_pe = nn.Sequential(nn.Linear(2048,1), nn.Sigmoid())
#         weights_init(self.linear_chronic_pe)
        
#         self.linear_acute_and_chronic_pe = nn.Sequential(nn.Linear(2048,1), nn.Sigmoid())
#         weights_init(self.linear_acute_and_chronic_pe)
        
#         self.linear_central_pe = nn.Sequential(nn.Linear(2048,1), nn.Sigmoid())
#         weights_init(self.linear_central_pe)
        
#         self.linear_leftsided_pe = nn.Sequential(nn.Linear(2048,1), nn.Sigmoid())
#         weights_init(self.linear_leftsided_pe)
        
#         self.linear_rightsided_pe = nn.Sequential(nn.Linear(2048,1), nn.Sigmoid())
#         weights_init(self.linear_rightsided_pe)
        
#         self.linear_rv_lv_ratio_gte_1 = nn.Sequential(nn.Linear(2048,1), nn.Sigmoid())
#         weights_init(self.linear_rv_lv_ratio_gte_1)
        
#         self.linear_rv_lv_ratio_lt_1 = nn.Sequential(nn.Linear(2048,1), nn.Sigmoid())
#         weights_init(self.linear_rv_lv_ratio_lt_1)
        
#         '''
#         informational features
#         '''
#         self.linear_qa_motion = nn.Sequential(nn.Linear(2048,1), nn.Sigmoid())
#         weights_init(self.linear_qa_motion)
        
#         self.linear_qa_contrast = nn.Sequential(nn.Linear(2048,1), nn.Sigmoid())
#         weights_init(self.linear_qa_contrast)
                
#         self.linear_true_filling_defect_not_pe = nn.Sequential(nn.Linear(2048,1), nn.Sigmoid())
#         weights_init(self.linear_true_filling_defect_not_pe)
        
#         self.linear_flow_artifact = nn.Sequential(nn.Linear(2048,1), nn.Sigmoid())
#         weights_init(self.linear_flow_artifact)

#     def forward(self, x):
#         x = self.backbone(x)
        
# #         print('x1 .shape', x.shape)

#         x_pe_present_on_image = self.linear_pe_present_on_image(x)
        
#         x_negative_exam_for_pe = self.linear_negative_exam_for_pe(x)
#         x_indeterminate = self.linear_indeterminate(x)
#         x_chronic_pe = self.linear_chronic_pe(x)
#         x_acute_and_chronic_pe = self.linear_acute_and_chronic_pe(x)
#         x_central_pe = self.linear_central_pe(x)
#         x_leftsided_pe = self.linear_leftsided_pe(x)
#         x_rightsided_pe = self.linear_rightsided_pe(x)
#         x_rv_lv_ratio_gte_1 = self.linear_rv_lv_ratio_gte_1(x)
#         x_rv_lv_ratio_lt_1 = self.linear_rv_lv_ratio_lt_1(x)
        
#         x_qa_motion = self.linear_qa_motion(x)
#         x_qa_contrast = self.linear_qa_contrast(x)
#         x_true_filling_defect_not_pe = self.linear_true_filling_defect_not_pe(x)
#         x_flow_artifact = self.linear_flow_artifact(x)
        
#         ys = []
#         for y_pred in [ x_pe_present_on_image, x_negative_exam_for_pe, x_indeterminate, x_chronic_pe, x_acute_and_chronic_pe, x_central_pe, x_leftsided_pe, x_rightsided_pe, x_rv_lv_ratio_gte_1, x_rv_lv_ratio_lt_1, x_qa_motion, x_qa_contrast, x_true_filling_defect_not_pe, x_flow_artifact]:
#             ys.append(y_pred)
#         return torch.cat(ys, 1)
                      

# model = WrapperModel(resnet101).to(device)

In [ ]:
model

In [ ]:
# df_train.loc[0][image_level_features+exam_level_features+informational_features]

In [ ]:
for name, param in model.named_parameters():
#     if not (name.startswith('backbone.layer3') or name.startswith('backbone.layer4') or name.startswith('backbone.Mixed_') or name.startswith('linear')):
    if not (name.startswith('backbone.layer4') or name.startswith('linear')):
        param.requires_grad = False
    else:
        param.requires_grad = True

In [ ]:
for name, param in model.named_parameters():
    print(name, param.requires_grad)

In [ ]:
df_train.head()

In [ ]:
df_train.loc[df_train.negative_exam_for_pe == 1]['pe_present_on_image'].values.sum()

In [ ]:
df_train.loc[df_train.indeterminate == 1][['qa_motion', 'qa_contrast']].values.sum()

In [ ]:
'''
Weights
'''
image_level_weights = [0.07361963]
exam_level_weights = [0.0736196319, 0.09202453988, 0.1042944785, 0.1042944785, 0.1877300613, 0.06257668712, 0.06257668712, 0.2346625767, 0.0782208589]

# Negative for PE	0.0736196319
# Indeterminate	0.09202453988
# Chronic	0.1042944785
# Acute & Chronic	0.1042944785
# Central PE	0.1877300613
# Left PE	0.06257668712
# Right PE	0.06257668712
# RV/LV Ratio >= 1	0.2346625767
# RV/LV Ratio < 1	0.0782208589



In [ ]:
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.0005, )

In [ ]:
# image level log loss (for one exam/study)
def image_level_loss(y_pred, y_real):
#     q_i = (y_preds[0].squeeze() > 0.5).sum().item()/y_real.shape[0]
#     return image_level_weights[0]*q_i*F.binary_cross_entropy(y_preds[0].squeeze(), y_real[:,0])
#     print(y_pred[0].squeeze())
#     print(y_real[:,0])
    return criterion(y_pred[0].squeeze(), y_real[:,0])

In [ ]:
def exam_level_loss(y_pred, y_real):
    total_loss = 0
#     for y_p, y_r, w in zip(y_preds[1:-4], y_real[:,1:-4].T, exam_level_weights):
#         loss = w*F.binary_cross_entropy(y_p.squeeze(), y_r)
#         total_loss += loss
    for y_p, y_r in zip(y_pred[1:-4], y_real[:,1:-4].T):
        loss = criterion(y_p.squeeze(), y_r)
        total_loss += loss
    return total_loss

In [ ]:
def compute_loss(y_pred, y_real):
    im_loss = image_level_loss(y_pred, y_real)
    ex_loss = exam_level_loss(y_pred, y_real)
    return im_loss+ex_loss

In [ ]:
import gc
gc.collect()

In [ ]:
def eval_on_sample(num_of_exams=8):
    losses = []
    for i in range(num_of_exams):
        rand_idx = np.random.randint(len(study_instances))
        X_eval, Y_eval = train_dataset[rand_idx]
        y_pred_eval = model(X_eval)
        loss_eval = compute_loss(y_pred_eval, Y_eval)
        losses.append(loss_eval)
    return losses


In [ ]:
# losses = eval_on_sample()

In [ ]:
# losses

In [ ]:
gc.collect()

In [ ]:
# model.load_state_dict(torch.load('./saved_resnet50'))
# # model.eval()

In [ ]:
exam_level_features

In [ ]:
torch.autograd.set_detect_anomaly(True)

In [ ]:
epochs = 20
steps_per_epoch = 1000

model.train()
for epoch in range(epochs):  
    print('Epoch Number:', epoch)
#     running_loss = 0.0
    for i, (x,y) in enumerate(train_dataloader):
    
#         try:
#         rand_idx = np.random.randint(len(study_instances))
#         X, Y = train_dataset[rand_idx]
#         print(X.shape)
#         optimizer.zero_grad()
#         y_pe_present_on_image, y_negative_exam_for_pe, y_indeterminate, y_chronic_pe, y_acute_and_chronic_pe, y_central_pe, y_leftsided_pe, y_rightsided_pe, y_rv_lv_ratio_gte_1, y_rv_lv_ratio_lt_1, y_qa_motion, y_qa_contrast, y_true_filling_defect_not_pe, y_flow_artifact = model(X)
        
#         image_level_loss = criterion(y_pe_present_on_image.squeeze(), Y[:,0])
        
#         exam_level_loss_1 = criterion(y_negative_exam_for_pe.squeeze(), Y[:,1])
#         exam_level_loss_2 = criterion(y_indeterminate.squeeze(), Y[:,2])
#         exam_level_loss_3 = criterion(y_chronic_pe.squeeze(), Y[:,3])
#         exam_level_loss_4 = criterion(y_acute_and_chronic_pe.squeeze(), Y[:,4])
#         exam_level_loss_5 = criterion(y_central_pe.squeeze(), Y[:,5])
#         exam_level_loss_6 = criterion(y_leftsided_pe.squeeze(), Y[:,6])
#         exam_level_loss_7 = criterion(y_rightsided_pe.squeeze(), Y[:,7])
#         exam_level_loss_8 = criterion(y_rv_lv_ratio_gte_1.squeeze(), Y[:,8])
#         exam_level_loss_9 = criterion(y_rv_lv_ratio_lt_1.squeeze(), Y[:,9])
        
#         total_loss = 2*image_level_loss + exam_level_loss_1 + exam_level_loss_2 + exam_level_loss_3 + exam_level_loss_4 + exam_level_loss_5 + exam_level_loss_6 + exam_level_loss_7 + exam_level_loss_8 + exam_level_loss_9
#         print(x[0].shape,x[1].shape, x[0].squeeze().shape, x[1].squeeze().shape)
#         print(x[0].shape, x[1].shape, 'BBBBB')
#         y = x[1].view(x[1].shape[0]*8,14)
#         x = x[0].view(x[1].shape[0]*8,3,224,224)
#         print(x[0].shape, x[1].shape, 'BBBBB')
#         y = x[1].view(-1, x[1].shape[2])
#         x = x[0].view(-1, x[0].shape[2], x[0].shape[3], x[0].shape[4])
#         print(y.shape, x.shape)
        optimizer.zero_grad()
        y_pred = model(x)
        total_loss = 0
#         print(y_pred, y)
#         optimizer.zero_grad()
        
        for i in range(y.shape[1]):
# #             print(y_pred[:,i])
# #             print()
# #             total_loss += criterion(y_pred[:,i], y[:,i])
# #             print(y_pred[:,i].shape, y[:,i].shape)
# #             print(y[:,i])
# #             loss_i = torch.autograd.Variable(criterion(y_pred[:,i], y[:,i]))
            loss_i = criterion(y_pred[:,i], y[:,i])
#             loss_i.backward(retain_graph=True)
            total_loss += loss_i
        total_loss.backward()
        optimizer.step()
#             total_loss = loss_i
#             print(i)
#         print(y_pred.shape, y.shape)
#         loss_i = criterion(y_pred, y)
#         loss_i.backward()
#         optimizer.step()
        
        #         print('AAAAAA', x.shape, y.shape)
#         print(y_pred[0].shape)
#         print(y_pred.shape)
#         total_loss = compute_loss(y_pred, y)
#         print(total_loss.requires_grad)
#         total_loss.backward()
#         optimizer.step()
        
        print('Loss:', total_loss.item())
# # #         print(y_preds[0].shape, Y.shape)
# #         loss = compute_loss(y_preds, Y)
#         print('Loss:', loss.item())
#         loss.backward()
#         optimizer.step()

print('Finished Training')

In [ ]:
# torch.save(model, './saved_resnet50')
torch.save(model.state_dict(), './saved_resnet50')

In [ ]:
model.eval()

In [ ]:
import gc
gc.collect()


# # del X
# del Y
# del y_preds

torch.cuda.empty_cache()

In [ ]:
# losses = eval_on_sample(6)

In [ ]:
# losses

In [ ]:
# def reshape_pred(pred_y):
#     ys = []
#     for y in pred_y:
#         ys.append(y)
#     return torch.cat(ys, 1)

In [ ]:
model.stage = 'test'

In [ ]:
# rand_idx = np.random.randint(len(study_instances))
for X, Y in train_dataloader:
    y_pred_eval = model(X)
    break

In [ ]:
# y_pred = reshape_pred(y_pred_eval)

In [ ]:
Y

In [ ]:
(y_pred_eval >= 0.5).type(torch.FloatTensor)

In [ ]:
y_pred_eval

In [ ]:
y_pred_eval

In [ ]:
y_pred_eval

In [ ]:
(y_pred >= 0.5).type(torch.FloatTensor)

In [ ]:
(y_pred >= 0.5).type(torch.FloatTensor)

In [ ]:
y_pred

In [ ]:
y_pred